In [1]:
! pip install tokenizer sentencepiece
! nvidia-smi

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/geantvert/.local/share/virtualenvs/kernl/bin/python -m pip install --upgrade pip' command.
Thu Oct 27 19:16:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:03:00.0  On |                  N/A |
| 42%   45C    P8    39W / 350W |    141MiB / 24576MiB |      1%      Default |
|                               |      

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import time
import torchdynamo
import torch
from kernl.model_optimization import optimize_model

In [3]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
input_ids = tokenizer(
    "translate English to French: The house in the woods is wonderful, can we buy it ?",
    return_tensors="pt",
    pad_to_multiple_of=8,
    padding=True,
).to("cuda")

In [5]:
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    for _ in range(10):
        output = model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
    torch.cuda.synchronize()
    start = time.perf_counter()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )
    torch.cuda.synchronize()
    latency_baseline = time.perf_counter() - start
    print(latency_baseline)
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

0.16218285400100285
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 


In [6]:
# share cuda pool between encoder/decoder
pool: (int, int) = torch.cuda.graph_pool_handle()


run_encoder = optimize_model(model.encoder, pool=pool)
run_decoder = optimize_model(model.decoder, pool=pool)

In [7]:
model.encoder.forward = run_encoder
model.decoder.forward = run_decoder

In [8]:
# warmup (IRL, encoder and decoder should be warmed each on their own)
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.perf_counter()
    model.generate(inputs=input_ids["input_ids"], min_length=22, max_length=22)
    print(time.perf_counter() - start)

105.08808013799717


In [9]:
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    for _ in range(10):
        model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
    torch.cuda.synchronize()
    start = time.perf_counter()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )
    torch.cuda.synchronize()
    latency_optimized = time.perf_counter() - start
    print(latency_optimized)
    print(f"{latency_baseline/latency_optimized:.1f}x speedup")
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

0.02129862899892032
7.6x speedup
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
